In [1]:
from engine.torchengine import AnalyticalSetSympy, FunctionSympy, EliminateAnalysisMergeResiduals, EliminateAnalysis, get_analysis_structure
from engine.torchdata import symbols, sp, load_vals, generate_optim_functions, load_file, print_formatted_table
import torch
from scipy.optimize import minimize
import numpy as np
from collections import namedtuple
import cyipopt
import json
np.set_printoptions(formatter={'float': lambda x: "{:0.2f}".format(x).rstrip('0').rstrip('.')})

In [7]:
alpha, d, A_solar, D_f, D_d, D_s, h_f, t_s, t_f, t_d, rho, rho_h, rho_w,V_d,g,F_B,F_W, m_platform,eta_solar, m_solar, m_struct, indices = symbols('alpha, d, A_solar, D_f, D_d, D_s, h_f, t_s, t_f, t_d, rho, rho_h, rho_w,V_d,g,F_B,F_W, m_platform,eta_solar, m_solar, m_struct')
idxrev = {var.item():key for key,var in indices.items()}

In [46]:
sets = {
    0: AnalyticalSetSympy(alpha*D_f, d, indices),
    1: AnalyticalSetSympy((1-alpha)*sp.pi*(D_f/2)**2, A_solar, indices),
    2: AnalyticalSetSympy(sp.pi/4*(D_f**2*h_f+D_s**2*t_s+D_d**2*t_d), V_d, indices),
    3: AnalyticalSetSympy(rho_w*V_d*g/1000, F_B, indices),
    4: AnalyticalSetSympy(F_B, F_W, indices),
    5: AnalyticalSetSympy(F_W*1000/g, m_platform, indices),
    6: AnalyticalSetSympy(eta_solar*A_solar, m_solar, indices),
    7: AnalyticalSetSympy(m_platform - m_solar, m_struct, indices),
    8: AnalyticalSetSympy((4/sp.pi*m_struct - D_f**2*t_f*rho - D_s**2*t_s*rho)/(D_d**2*rho_h), t_d, indices)
}

In [47]:
equality_constraints = EliminateAnalysisMergeResiduals()
inequality_constraints = EliminateAnalysisMergeResiduals(functions=[
    FunctionSympy(h_f-0.9*t_f, indices),
    FunctionSympy(D_s-0.9*D_f, indices),
    FunctionSympy(D_s-0.9*D_d, indices)])
objective = FunctionSympy(m_platform, indices)

In [48]:
order = list(sorted(sets.keys()))
A = []
B = [elt for elt in order if elt not in A]
analyses = {key: s.analysis for key,s in sets.items()}
residuals = {key: s.residual for key,s in sets.items()}

# # Elimination option
# order = list(sorted(sets.keys()))
# structure_in, structure_out, full_structure = get_analysis_structure(
#     [sets[idx].analysis.structure_full for idx in order])
# feedback = {int(elt) for elt in structure_in}.intersection({int(elt) for elt in structure_out})
# A = [elt for elt in order if int(sets[elt].analysis.structure[1]) not in feedback]
# B = [elt for elt in order if elt not in A]
# new_order = A+B
# analyses = {key: s.analysis for key,s in sets.items()}
# residuals = {key: s.residual for key,s in sets.items()}

In [49]:
# Merge residuals only
R = EliminateAnalysisMergeResiduals(functions=[residuals[idx] for idx in B])
P = EliminateAnalysis([analyses[idx] for idx in A], [objective,R,equality_constraints,inequality_constraints])
objective_idx,residual_idx,equality_idx,inequality_idx = 0,1,2,3

In [50]:
x0 = load_vals('pearl_params', indices=indices)
x0 = load_vals('results', indices=indices, x0=x0)

In [51]:
[np.round(elt.numpy(),7) for elt in P(x0)]

[array([0]),
 array([0.01, 0.01, 0, 0, 0, 0, 0, 0, -0.31]),
 array([], dtype=float32),
 array([-0.56, -0.01, -0.01])]

In [52]:
optim_indices = P.structure[0]
p = load_file('pearl_params')
pindices = [val for key,val in indices.items() if str(key) in p.keys()]
optim_indices = torch.tensor([idx for idx in P.structure[0] if idx not in pindices])

In [57]:
# MAKE SURE TO SET INEQUALITY DIRECTION, it is different for scipy and ipopt
xguess, obj_function, ineq_function, eq_function, dobj, dineq, deq = generate_optim_functions(P,
    optim_indices, x0, inequality_direction='negative-null', 
    objective=objective_idx, residuals=residual_idx, equalities=equality_idx, inequalities=inequality_idx)
ineqlen = len(ineq_function(xguess))
eqlen = len(eq_function(xguess))
constraints = [{'type': 'eq', 'fun': eq_function, 'jac': deq}] if eqlen >= 1 else []
constraints.append({'type': 'ineq', 'fun': ineq_function, 'jac': dineq}) if ineqlen >= 1 else []
bounds = {
    'D_f': (0.1,10),
    'D_d': (0.1,10),
    'D_s': (0.1,10),
    't_s': (0.1,10),
    't_d': (0.1,10),
    't_f': (0.1,10),
    'h_f': (0.1,10)
}
bnds_problem = [bounds.get(str(idxrev[elt.item()]), (0, None)) for elt in optim_indices]

In [58]:
[(idxrev[elt.item()],xguess[idx]) for idx,elt in enumerate(optim_indices)]

[(m_platform, 7.022616011390322),
 (D_f, 0.11111111085640367),
 (d, 0.0055555555428201675),
 (A_solar, 0.009211459898849092),
 (D_d, 0.11111111112688439),
 (D_s, 0.1),
 (V_d, 0.0070226160113903285),
 (h_f, 0.1),
 (t_d, 0.10000000001439217),
 (t_s, 0.6472336623620831),
 (F_B, 0.06882163691162516),
 (F_W, 0.06882163691162516),
 (m_solar, 0.09211459898849092),
 (m_struct, 6.930501412401831),
 (t_f, 0.11111111372185745)]

In [55]:
# Solve the optimization problem
xsol = minimize(obj_function, xguess, bounds=bnds_problem, jac=dobj, 
                constraints=constraints, options={'maxiter': 500}, method='SLSQP')

In [56]:
xsol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 7.022616011390322
       x: [ 7.023e+00  1.111e-01 ...  6.931e+00  1.111e-01]
     nit: 5
     jac: [ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
    nfev: 5
    njev: 5

In [59]:
def all_constraints(x):
    return np.concatenate([ineq_function(x), eq_function(x)])

def all_constraints_jac(x):
    if eqlen == 0:
        return dineq(x)
    return np.concatenate([dineq(x), deq(x)], axis=0)

OptProblem = namedtuple('OptProblem', ['objective', 'constraints', 'gradient', 'jacobian', 'intermediate'])

lb,ub = zip(*bnds_problem)
cl = np.concatenate([-np.inf*np.ones(ineqlen), np.zeros(eqlen)])
cu = np.concatenate([np.zeros(ineqlen), np.zeros(eqlen)])

storeiter = [0]

def logiter(alg_mod, iter_count, obj_value, inf_pr, inf_du, mu, d_norm, regularization_size, alpha_du, alpha_pr, ls_trials):
    storeiter[0] = iter_count

# define the problem
probinfo = OptProblem(obj_function, all_constraints, dobj, all_constraints_jac, logiter)

prob = cyipopt.Problem(n=len(xguess), m=len(cu), lb=lb, ub=ub, cl=cl, cu=cu, 
                       problem_obj=probinfo)
prob.add_option('max_iter', 8000)

In [60]:
xsol, optinfo = prob.solve(xguess)

In [61]:
optinfo

{'x': array([7.02, 0.11, 0.01, 0.01, 0.11, 0.1, 0.01, 0.1, 0.1, 0.65, 0.07,
        0.07, 0.09, 6.93, 0.11]),
 'g': array([0, 0, 0, 0, 0, -0, 0, 0, 0, -0, 0, 0]),
 'obj_val': 7.022612219255623,
 'mult_g': array([25.14, 11.17, 129.28, -0, 33.33, -2333.33, -238.1, -238.1, -2.33,
        3.33, -3.33, -87.27]),
 'mult_x_L': array([0, 0, 0, 0, 0, 140.45, 0, 2.51, 64.64, 0, 0, 0, 0, 0, 0]),
 'mult_x_U': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'status': 0,
 'status_msg': b'Algorithm terminated successfully at a locally optimal point, satisfying the convergence tolerances (can be specified by options).'}

In [62]:
Aelim = EliminateAnalysis([analyses[idx] for idx in A])
yout = Aelim(x0)

In [63]:
print_formatted_table([yout, x0], indices, idxrev, subset=optim_indices)

m_platform D_f   d        A_solar  D_d   D_s V_d      h_f t_d t_s   F_B   F_W   m_solar m_struct t_f  
7.023      0.111 5.56e-03 9.21e-03 0.111 0.1 7.02e-03 0.1 0.1 0.647 0.069 0.069 0.092   6.93     0.111
7.023      0.111 5.56e-03 9.21e-03 0.111 0.1 7.02e-03 0.1 0.1 0.647 0.069 0.069 0.092   6.93     0.111


In [64]:
results = {str(idxrev[elt.item()]): yout[elt].item() for elt in optim_indices}

In [38]:
name = 'results.json'
json_string = json.dumps(results, indent=4)
with open('../applications/data/{}'.format(name), 'w') as file:
    file.write(json_string)
